In [14]:
import json, uuid, os
from datetime import datetime, timezone
from pymongo import MongoClient
from dotenv import load_dotenv

# --- Load environment variables ---
load_dotenv()
uri = os.getenv("MONGODB_URI")
database_name = os.getenv("MONGODB_DATABASE")
collection_name = "sales_invoices"

# --- Connect to MongoDB ---
client = MongoClient(uri)
db = client[database_name]
col = db[collection_name]

# --- Find latest EMPTY document (items missing OR empty array) ---
latest_empty = col.find_one(
    {
        "$or": [
            {"items": {"$exists": False}},
            {"items": {"$size": 0}}
        ]
    },
    sort=[("created", -1)]
)

if latest_empty:
    print("✅ Reusing latest EMPTY form (no new document created).")
    print(json.dumps(latest_empty, indent=2, default=str))
else:
    # --- Find latest existing document for inheritance ---
    latest_doc = col.find_one(sort=[("created", -1)])

    prev_qr = latest_doc.get("current_form_qr_code", "") if latest_doc else ""
    prev_tin = latest_doc.get("tin", "") if latest_doc else ""
    prev_location = latest_doc.get("location", "") if latest_doc else ""

    # --- Build new empty form ---
    new_doc = {
        "id": str(uuid.uuid4()),
        "previous_form_qr_code": prev_qr,
        "current_form_qr_code": str(uuid.uuid4())[:8],
        "tin": prev_tin,
        "location": prev_location,
        "created": datetime.now(timezone.utc).isoformat(),
        "date": datetime.now(timezone.utc).date().isoformat()
    }

    res = col.insert_one(new_doc)
    new_doc["_id"] = str(res.inserted_id)

    print("✅ Created new EMPTY form (no existing empty one found).")
    print(json.dumps(new_doc, indent=2, default=str))


✅ Reusing latest EMPTY form (no new document created).
{
  "_id": "68ed1b27c3992aaea60563f1",
  "id": "5812d170-c0fb-4a77-a0f1-263616dc7a67",
  "previous_form_qr_code": "20251012",
  "current_form_qr_code": "a1b36b5a",
  "tin": "123-456-789",
  "location": "JEF Gas Station \u2013 Sikatuna Branch",
  "created": "2025-10-13T15:30:47.572217+00:00",
  "date": "2025-10-13"
}
